<a href="https://colab.research.google.com/github/leesolhahaha/study/blob/main/CNN%EC%9D%84_%ED%86%B5%ED%95%9C_%EA%B8%B0%EC%B4%88_%EC%9D%8C%EC%84%B1_%EB%B6%84%EB%A5%98.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
## 학습 실패

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import librosa
import librosa.display as dsp
from IPython.display import Audio

In [ ]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import os

In [ ]:
import torch

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu') #GPU 할당

In [ ]:
## 모델의 재현성을 위하여 random seed 고정
import random

def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

seed_everything(929)

In [ ]:
## 분석하려는 데이터를 가져오기
import pandas as pd
train = pd.read_csv('/content/drive/MyDrive/엑셀파일/이진분류/이진학습.csv')
train.head()

,fold,slice_file_name,class_name
0,[원천]4.폭력범죄,4.폭력범죄_844421_label.wav,위급
1,[원천]4.폭력범죄,4.폭력범죄_291855_label.wav,위급
2,[원천]4.폭력범죄,4.폭력범죄_2234_label.wav,위급
3,[원천]4.폭력범죄,4.폭력범죄_284456_label.wav,위급
4,[원천]4.폭력범죄,4.폭력범죄_841165_label.wav,위급


In [ ]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13784 entries, 0 to 13783
Data columns (total 3 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   fold             13784 non-null  object
 1   slice_file_name  13784 non-null  object
 2   class_name       13784 non-null  object
dtypes: object(3)
memory usage: 323.2+ KB


In [ ]:
## 음성 데이터 load
data, sample_rate = librosa.load('/content/drive/MyDrive/분류/[원천]1.강제추행(성범죄)/train/1.강제추행(성범죄)_105083_label.wav', sr = 16000)
print('sample_rate:', sample_rate, ', audio shape:', data.shape)
print('length:', data.shape[0]/float(sample_rate), 'secs')

sample_rate: 16000 , audio shape: (37439,)
length: 2.3399375 secs


In [ ]:
## 디지털 신호와 라벨을 포함한 데이터프레임 생성
def train_dataset():
    folder = "/content/drive/MyDrive/분류/"
    dataset = []
    for index, row in tqdm(train.iterrows(), total=len(train), colour='green'):
        file_name = row['slice_file_name']
        abs_file_path = os.path.join(folder, str(row['fold']),'train', file_name)
        data, sr = librosa.load(abs_file_path, sr=16000)
        class_label = str(row['class_name'])  # Convert label to string
        dataset.append([data, class_label])

    print("Dataset 생성 완료")
    return pd.DataFrame(dataset, columns=['data', 'label'])

In [ ]:
def test_dataset():
    test = pd.read_csv("/content/drive/MyDrive/엑셀파일/이진분류/이진테스트.csv")  # Replace with the actual path

    folder = '/content/drive/MyDrive/분류/'
    dataset = []
    for index, row in tqdm(test.iterrows(), total=len(test), colour='green'):
        file_name = row['slice_file_name']
        abs_file_path = os.path.join(folder, str(row['fold']),'test', file_name)
        data, sr = librosa.load(abs_file_path, sr=16000)

        dataset.append([data, file_name])

    print("Dataset 생성 완료")
    return pd.DataFrame(dataset, columns=['data', 'file_name'])

In [ ]:
train_wav = train_dataset()
test_wav = test_dataset()

100%|██████████| 13784/13784 [1:05:06<00:00,  3.53it/s]


Dataset 생성 완료


100%|██████████| 1731/1731 [08:11<00:00,  3.52it/s]

Dataset 생성 완료


In [ ]:
train_wav.head()

,data,label
0,"[-0.0030573756, -0.004531581, -0.0032278895, -...",위급
1,"[0.0005252836, 0.0008690482, 0.0005630741, 0.0...",위급
2,"[6.48703e-05, 0.0001558671, 0.000382535, 0.000...",위급
3,"[4.3919736e-05, -8.4328385e-06, -0.00021260773...",위급
4,"[-0.00035066914, -0.00019957856, 0.0002971351,...",위급


In [ ]:
test_wav.head()

,data,file_name
0,"[-0.00016276332, -0.0001991705, -5.7802405e-05...",1.강제추행(성범죄)_147322_label.wav
1,"[0.00014566121, 0.0003229993, 0.00027627402, 0...",1.강제추행(성범죄)_125108_label.wav
2,"[-0.00018318684, -0.00025979185, -0.0001769727...",1.강제추행(성범죄)_27452_label.wav
3,"[-1.625897e-05, -4.7569094e-05, 2.2155868e-05,...",1.강제추행(성범죄)_148012_label.wav
4,"[-0.00014499975, -0.00018902586, -2.2587865e-0...",1.강제추행(성범죄)_138886_label.wav


In [ ]:
## 데이터프레임 csv파일로 저장
## 이전 코드에서 데이터프레임 생성하는데 오래 걸리기 때문
train_wav.to_csv('/content/drive/MyDrive/엑셀파일/cnn 데이터프레임 파일/cnn_train.csv')
test_wav.to_csv('/content/drive/MyDrive/엑셀파일/cnn 데이터프레임 파일/cnn_test.csv')

In [ ]:
train_x = np.array(train_wav.data)
test_x = np.array(test_wav.data)

In [ ]:
# 음성의 길이 중 가장 작은 길이를 구합니다.

def get_mini(data):

    mini = 9999999
    for i in data:
        if len(i) < mini:
            mini = len(i)

    return mini

train_mini = get_mini(train_x)
test_mini = get_mini(test_x)

#음성들의 길이를 맞춰줍니다.

mini = np.min([train_mini, test_mini])

In [ ]:
print('가장 작은 길이 :', mini)

가장 작은 길이 : 7680


In [ ]:
## data의 길이를 가장 작은 길이에 맞춰 잘라준다.
def set_length(data, d_mini):

    result = []
    for i in data:
        result.append(i[:d_mini])
    result = np.array(result)

    return result

train_x = set_length(train_x, mini)
test_x = set_length(test_x, mini)

In [ ]:
## 전처리 된 음성 데이터 특징 모양
print('train :', train_x.shape)
print('test :', test_x.shape)

train : (13784, 7680)
test : (1731, 7680)


In [ ]:
extracted_features = librosa.feature.mfcc(y=train_x[0], sr=16000, n_mfcc=40)
extracted_features.shape ## 출력값은 (n_mfcc, time_step) 으로 구성

(40, 16)

In [ ]:
## train 셋 데이터 전체를 MFCC 알고리즘을 이용하여 특징추출
def preprocess_dataset(data):
    mfccs = []
    for i in data:
        extracted_features = librosa.feature.mfcc(y=i,
                                              sr=16000,
                                              n_mfcc=40)
        mfccs.append(extracted_features)

    return mfccs

In [ ]:
train_mfccs = preprocess_dataset(train_x)
train_mfccs = np.array(train_mfccs)
train_mfccs = train_mfccs.reshape(-1, train_mfccs.shape[1], train_mfccs.shape[2], 1)
#test_x = test_x.reshape(-1, test_x.shape[1], test_x.shape[2], 1)

In [ ]:
np.array(train_mfccs).shape

(13784, 40, 16, 1)

In [ ]:
## dataset을 구성하기 위한 CustomDataset 클래스 생성
import torchvision.datasets as datasets # 데이터셋 집합체
import torchvision.transforms as transforms # 변환 툴

from torch.utils.data import DataLoader # 학습 및 배치로 모델에 넣어주기 위한 툴
from torch.utils.data import DataLoader, Dataset

class CustomDataset(Dataset):
    def __init__(self, X, y, train_mode=True, transforms=None):
        self.X = X
        self.y = y
        self.train_mode = train_mode
        self.transforms = transforms

    def __getitem__(self, index):
        data = self.X[index]
        label = self.y[index]

        if self.transforms is not None:
            data = self.transforms(data)

        if self.train_mode:
            return data, label
        else:
            return data

    def __len__(self):
        return len(self.X)

def convert_labels_to_int(labels):
    return [label_to_int[label] for label in labels]

In [ ]:
label_to_int = {'위급': 0, '단순소음': 1}

In [ ]:
## 학습 데이터 / 검증 데이터 (3:1 비율로 나누기)
train_X = train_mfccs[:300]
vali_X = train_mfccs[300:]

train_y = train_wav.label[:300]
vali_y = train_wav.label[300:].reset_index(drop = True)

In [ ]:
## batch기반의 딥러닝모델 학습을 위해서 mini batch 만들기
# 에포크 설정
num_epochs = 100

# 배치 사이즈 설정
batch_size = 10

#만든 train dataset를 DataLoader에 넣어 batch 만들기
train_dataset = CustomDataset(X=train_X, y=train_y)
train_loader = DataLoader(train_dataset, batch_size = batch_size, shuffle=True)

vali_dataset = CustomDataset(X=vali_X, y=vali_y)
vali_loader = DataLoader(vali_dataset, batch_size = batch_size, shuffle=False)

In [ ]:
train_batches = len(train_loader)
vali_batches = len(vali_loader)

print('/ total train batches :', train_batches)
print('/ total valid batches :', vali_batches)

/ total train batches : 30
/ total valid batches : 1349


In [ ]:
## 모델 구조 정의
from tqdm.auto import tqdm
import torch.nn as nn # 신경망들이 포함됨

class CNNclassification(torch.nn.Module):
    def __init__(self):
        super(CNNclassification, self).__init__()
        self.layer1 = torch.nn.Sequential(
            nn.Conv2d(40, 10, kernel_size=2, stride=1, padding=1), #cnn layer
            nn.ReLU(), #activation function
            nn.MaxPool2d(kernel_size=2, stride=2)) #pooling layer

        self.layer2 = torch.nn.Sequential(
            nn.Conv2d(10, 100, kernel_size=2, stride=1, padding=1), #cnn layer
            nn.ReLU(), #activation function
            nn.MaxPool2d(kernel_size=2, stride=2)) #pooling layer

        self.layer3 = torch.nn.Sequential(
            nn.Conv2d(100, 200, kernel_size=2, stride=1, padding=1), #cnn layer
            nn.ReLU(), #activation function
            nn.MaxPool2d(kernel_size=2, stride=2)) #pooling layer

        self.layer4 = torch.nn.Sequential(
            nn.Conv2d(200, 300, kernel_size=2, stride=1, padding=1), #cnn layer
            nn.ReLU(), #activation function
            nn.MaxPool2d(kernel_size=2, stride=2)) #pooling layer

        self.fc_layer = nn.Sequential(
            nn.Linear(300, 10) #fully connected layer(ouput layer)
        )

    def forward(self, x):

        x = self.layer1(x) #1층

        x = self.layer2(x) #2층

        x = self.layer3(x) #3층

        x = self.layer4(x) #4층

        x = torch.flatten(x, start_dim=1) # N차원 배열 -> 1차원 배열

        out = self.fc_layer(x)
        return out

In [ ]:
## 모델 학습
import torch.optim as optim # 최적화 알고리즘들이 포함힘

model = CNNclassification().to(device)
criterion = torch.nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.SGD(params = model.parameters(), lr = 1e-3 )
scheduler = None

In [ ]:
model(torch.rand(10, 40, 16, 1).to(device))

tensor([[-0.0437,  0.0300, -0.0339, -0.0165,  0.0755,  0.0170,  0.0379,  0.0027,
         -0.0174,  0.0802],
        [-0.0431,  0.0306, -0.0309, -0.0142,  0.0749,  0.0181,  0.0355,  0.0039,
         -0.0154,  0.0819],
        [-0.0408,  0.0307, -0.0338, -0.0173,  0.0731,  0.0213,  0.0337,  0.0029,
         -0.0203,  0.0794],
        [-0.0405,  0.0286, -0.0326, -0.0173,  0.0742,  0.0198,  0.0354,  0.0011,
         -0.0160,  0.0842],
        [-0.0427,  0.0265, -0.0339, -0.0175,  0.0741,  0.0173,  0.0342,  0.0034,
         -0.0165,  0.0804],
        [-0.0444,  0.0311, -0.0333, -0.0155,  0.0740,  0.0178,  0.0341,  0.0039,
         -0.0164,  0.0808],
        [-0.0420,  0.0306, -0.0330, -0.0157,  0.0700,  0.0212,  0.0323,  0.0013,
         -0.0189,  0.0779],
        [-0.0426,  0.0292, -0.0321, -0.0186,  0.0720,  0.0186,  0.0361,  0.0020,
         -0.0164,  0.0806],
        [-0.0409,  0.0289, -0.0337, -0.0185,  0.0710,  0.0197,  0.0370,  0.0022,
         -0.0181,  0.0799],
        [-0.0430,  

In [ ]:
## train을 학습 시켜 vaildation으로 평가하는 메소드
from tqdm.auto import tqdm

def train(model, optimizer, train_loader, scheduler, device):
    model.to(device)
    n = len(train_loader)
    best_acc = 0

    for epoch in range(1, num_epochs):  # 에포크 설정
        model.train() # 모델 학습
        running_loss = 0.0

        for wav, labels in tqdm(iter(train_loader)):
            wav = wav.to(device)
            labels = convert_labels_to_int(labels)  # 문자열 레이블을 정수로 변환
            labels = torch.tensor(labels).to(device)

            optimizer.zero_grad() ## 배치마다 optimizer 초기화

            logit = model(wav) ## 예측값 산출
            loss = criterion(logit, labels) ## 손실함수 계산

            ## 역전파
            loss.backward() ## 손실함수 기준 역전파
            optimizer.step() ## 가중치 최적화
            running_loss += loss.item()

        print('[%d] Train loss: %.10f' %(epoch, running_loss / len(train_loader)))

        if scheduler is not None:
            scheduler.step()


        #Validation set 평가
        model.eval() #evaluation 과정에서 사용하지 않아야 하는 layer들을 알아서 off 시키도록 하는 함수
        vali_loss = 0.0
        correct = 0

        with torch.no_grad(): #파라미터 업데이트 안하기 때문에 no_grad 사용
            for wav, labels in tqdm(iter(vali_loader)):
                wav = wav.to(device)
                labels = convert_labels_to_int(labels)  # 문자열 레이블을 정수로 변환
                labels = torch.tensor(labels).to(device)
                logit = model(wav)
                vali_loss += criterion(logit, labels)
                pred = logit.argmax(dim=1, keepdim=True) #10개의 class중 가장 값이 높은 것을 예측 label로 추출
                correct += pred.eq(labels.view_as(pred)).sum().item() #예측값과 실제값이 맞으면 1 아니면 0으로 합산

        vali_acc = 100 * correct / len(vali_loader.dataset)
        print('Validation Loss: {:.4f}, Validation Accuracy: {:.0f}%'.format(
            vali_loss / len(vali_loader), vali_acc))

        #베스트 모델 저장
        if best_acc < vali_acc:
            best_acc = vali_acc
            torch.save(model.state_dict(), '/content/drive/MyDrive/데이터셋/best_model2.pth') #이 디렉토리에 best_model.pth을 저장
            print('Model Saved.')

In [ ]:
train(model, optimizer, train_loader, scheduler, device)

  0%|          | 0/30 [00:00<?, ?it/s]

[1] Train loss: 0.0031620756


  0%|          | 0/1349 [00:00<?, ?it/s]

Validation Loss: 10.0013, Validation Accuracy: 29%
Model Saved.


  0%|          | 0/30 [00:00<?, ?it/s]

...

Validation Loss: 16.9388, Validation Accuracy: 29%


  0%|          | 0/30 [00:00<?, ?it/s]

[99] Train loss: 0.0000831139


  0%|          | 0/1349 [00:00<?, ?it/s]

Validation Loss: 16.9615, Validation Accuracy: 29%
